In [1]:
!pip install -q -U "tensorflow_text"

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [4]:
import pandas as pd
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

In [5]:
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [6]:
train = train.drop(columns=['keyword','id','location'])
test = test.drop(columns=['keyword','id','location'])

In [7]:
train['target'].value_counts()

,count
target,
0,4342
1,3271


In [8]:
X_train = train['text']
y_train = train['target']
X_test = test['text']

In [9]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [10]:
bert_preprocess = hub.KerasLayer(preprocess_url)
bert_encoder = hub.KerasLayer(encoder_url)

In [12]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

model = tf.keras.Model(inputs=[text_input], outputs = [l])

ValueError: Exception encountered when calling layer 'keras_layer_2' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_2' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=string, sparse=False, name=text>
  • training=None

In [14]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text  # Ensure this is installed

# Load BERT Preprocessing and Encoder Models
bert_preprocess = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3", name="bert_preprocess"
)
bert_encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3", name="bert_encoder"
)

# Define Input Layer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

# Wrap the preprocessing and encoding in a tf.function
# This allows TensorFlow to handle the symbolic input correctly
@tf.function(input_signature=[tf.TensorSpec(shape=(), dtype=tf.string)])
def bert_encode(text_input):
    preprocessed_text = bert_preprocess(text_input)  # Outputs a dict with keys: input_word_ids, input_mask, segment_ids
    outputs = bert_encoder(preprocessed_text)  # Encoder expects this dictionary
    return outputs['pooled_output']

# Use the wrapped function in the model definition
l = tf.keras.layers.Dropout(0.1, name="dropout")(bert_encode(text_input))
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Define and Compile Model
model = tf.keras.Model(inputs=[text_input], outputs=[l])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print Model Summary
model.summary()

TypeError: Binding inputs to tf.function failed due to `A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
`. Received args: (<KerasTensor shape=(None,), dtype=string, sparse=False, name=text>,) and kwargs: {} for signature: (text_input: TensorSpec(shape=(), dtype=tf.string, name=None)).

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=20)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
y_pred = pd.Series(y_predicted,name='target')

In [ ]:
test = pd.read_csv("/content/test.csv")

In [ ]:
y_pre = pd.concat([pd.Series(test['id']),pd.Series(y_predicted,name='target')],axis=1)

In [ ]:
y_pre.to_csv('sub12.csv',index=False)